In [2]:
import os,sys,string
from vtk import *
 

In [3]:
tse = vtkTimeSourceExample()
ex = tse.GetExecutive()
tse.UpdateInformation()
 
#inspect available time range and time steps
print ex.GetOutputInformation()
 
#make it grow because bounds are easy to inspect
# tse.SetGrowing(1)
 
ex.SetUpdateTimeStep(0,0.0)
tse.Update()
print(tse.GetOutput().GetBounds())
 
#pick some other time inside the time range
ex.SetUpdateTimeStep(0,0.5)
tse.Update()
print(tse.GetOutput().GetBounds())
 
grid = tse.GetOutput()
print grid

vtkInformationVector (0x173c620)
  Debug: Off
  Modified Time: 398
  Reference Count: 2
  Registered Events: (none)
  Number of Information Objects: 1
  Information Objects:
    vtkInformation(0x173cec0):
      Debug: Off
      Modified Time: 475
      Reference Count: 1
      Registered Events: (none)
      PRODUCER: vtkCompositeDataPipeline(0x1738120) port 0
      CAN_HANDLE_PIECE_REQUEST: 1
      UPDATE_NUMBER_OF_GHOST_LEVELS: 0
      TIME_STEPS: 0 0.111111 0.222222 0.333333 0.444444 0.555556 0.666667 0.777778 0.888889 1
      TIME_RANGE: 0 1
      UPDATE_PIECE_NUMBER: 0
      UPDATE_EXTENT_INITIALIZED: 0
      UPDATE_NUMBER_OF_PIECES: 1
      DATA_OBJECT: vtkUnstructuredGrid(0x13410d0)


(0.0, 1.0, 0.0, 1.0, 0.0, 1.0)
(0.0, 1.0, 0.0, 1.0, 0.0, 1.0)
vtkUnstructuredGrid (0x13410d0)
  Debug: Off
  Modified Time: 702
  Reference Count: 2
  Registered Events: (none)
  Information: 0x1738730
  Data Released: False
  Global Release Data: Off
  UpdateTime: 707
  Field Data:
    Debug: Off


In [5]:
tri = vtkDataSetTriangleFilter()
tri.SetInputDataObject(grid)
tri.SetTetrahedraOnly(1)
tri.Update()
output = tri.GetOutput()
 

In [7]:
iss = output.GetPointData().SetActiveScalars("Point Label")
# iss = gridMapper.GetInput().GetCellData().SetActiveScalars(options.scalarName)
assert(iss>-1)
 
drange = [0, 1]
 


In [8]:
# Create transfer mapping scalar value to opacity.
opacityFunction = vtkPiecewiseFunction()
opacityFunction.AddPoint(drange[0], 0.0)
opacityFunction.AddPoint(drange[1], 1.0)
 
# Create transfer mapping scalar value to color.
colorFunction = vtkColorTransferFunction()
colorFunction.SetColorSpaceToHSV()
colorFunction.HSVWrapOff()
colorFunction.AddRGBPoint(drange[0], 0.0, 0.0, 1.0)
colorFunction.AddRGBPoint(drange[1], 1.0, 0.0, 0.0)
 
volumeProperty = vtkVolumeProperty()
volumeProperty.SetScalarOpacity(opacityFunction)
volumeProperty.SetColor(colorFunction)
volumeProperty.ShadeOff()
volumeProperty.SetInterpolationTypeToLinear()
# volumeProperty.SetScalarOpacityUnitDistance(options.unit)


In [12]:
volumeMapper = vtkUnstructuredGridVolumeRayCastMapper()
# volumeMapper = vtkUnstructuredGridVolumeZSweepMapper()
# volumeMapper = vtkProjectedTetrahedraMapper()
# volumeMapper.SetBlendModeToMaximumIntensity()
volumeMapper.SetInputData(output)


In [16]:
 
volume = vtkVolume()
volume.SetMapper(volumeMapper)
volume.SetProperty(volumeProperty)
 
# create a rendering window and renderer
renderer = vtkRenderer()
renderer.SetBackground(0,0,0)
 
window = vtkRenderWindow()
window.SetSize(512,512)
window.AddRenderer(renderer)
 
interactor = vtkRenderWindowInteractor()
interactor.SetRenderWindow(window)
 
style = vtkInteractorStyleTrackballCamera();
interactor.SetInteractorStyle(style);
 
renderer.AddVolume(volume)
 
scalarBar = vtkScalarBarActor()
scalarBar.SetLookupTable(colorFunction);
scalarBar.SetOrientationToVertical();
scalarBar.SetPosition( 0.85, 0.7 );
scalarBar.SetPosition2( 0.1, 0.3 );
propT = vtkTextProperty()
propL = vtkTextProperty()
propT.SetFontFamilyToArial()
propT.ItalicOff()
propT.BoldOn()
propL.BoldOff()
scalarBar.SetTitleTextProperty(propT);
scalarBar.SetLabelTextProperty(propL);
scalarBar.SetLabelFormat("%5.2f")
renderer.AddActor(scalarBar)
 
# setup the text and add it to the window
textActor = vtkTextActor()
textActor.GetTextProperty().SetFontSize(12)
textActor.SetPosition2(10, 40)
renderer.AddActor2D(textActor)
textActor.SetInput("time = ")
textActor.GetTextProperty().SetColor(1.0,1.0,1.0)
 
renderer.ResetCameraClippingRange()
renderer.ResetCamera()
 
counter = 1
time = 0
while time<=1:
    print "time = ", time
    textActor.SetInput("time = %g" % time)

    window.Render()
 
    # TODO FIXME if this block is not here than the volume renders wrongly
    renderer.RemoveVolume(volume)
    # del volume
    volume = vtkVolume()
    volume.SetMapper(volumeMapper)
    volume.SetProperty(volumeProperty)
    renderer.AddVolume(volume)
 
    counter = counter + 1
    time = time + 1./10
    ex.SetUpdateTimeStep(0,time)
    tse.Modified()
    #print grid.GetPointData().GetScalars().GetRange()
    print grid.GetPointData()
    # print grid.GetPointData().GetScalars()
 
# while True:
    # window.Render()

time =  0
vtkPointData (0x172d400)
  Debug: Off
  Modified Time: 676
  Reference Count: 2
  Registered Events: (none)
  Number Of Arrays: 5
  Array 0 name = Point Value
  Array 1 name = Point Label
  Array 2 name = Point X
  Array 3 name = Point Y
  Array 4 name = Point Z
  Number Of Components: 5
  Number Of Tuples: 8
  Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
  Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
  Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
  Scalars: (none)
  Vectors: (none)
  Normals: (none)
  TCoords: (none)
  Tensors: (none)
  GlobalIds: 
    Debug: Off
    Modified Time: 661
    Reference Count: 2
    Registered Events: (none)
    Name: Point Label
    Data type: idtype
    Size: 8
    MaxId: 7
    NumberOfComponents: 1
    Information: 0x28c0810
      Debug: Off
      Modified Time: 5665
      Reference Count: 1
      Registered Events: (none)
      PER_COMPONENT: vtkInformationVector(0x28c0a10)
    Name: Point Label
    Number Of Components: 1
    Number Of Tuples: 8
    Size: 8


In [ ]:
l